In [ ]:
from pyDatalog import pyDatalog

pyDatalog.create_terms('X,Y,Z, manager, indirect_manager')

indirect_manager(X,Y) <=  manager(X,Y)
indirect_manager(X,Y) <= manager(X,Z) & indirect_manager(Z,Y)


In [ ]:
+manager('John', 'Mary')
+manager('Mary', 'Jerry')

In [ ]:
t = manager(X, Y)
type(t)
for q in t:
    print(q)

In [ ]:
indirect_manager(X, Y)

In [ ]:
from pyDatalog import pyDatalog
pyDatalog.create_terms('square, adjacent, R1, C1, R2, C2, R, C')

for i in range(1,4):
    for j in range(1,4):
        +square(i,j)

In [ ]:
adjacent(R1, C1, R2, C2 ) <= square(R1, C1) & square(R2, C2) & (R1 == R2 + 1) & (C1 == C2)
adjacent(R1, C1, R2, C2 ) <= square(R1, C1) & square(R2, C2) & (R1 == R2 - 1) & (C1 == C2)
adjacent(R1, C1, R2, C2 ) <= square(R1, C1) & square(R2, C2) & (C1 == C2 + 1) & (R1 == R2)
adjacent(R1, C1, R2, C2 ) <= square(R1, C1) & square(R2, C2) & (C1 == C2 - 1) & (R1 == R2)

In [ ]:
adjacent(2,2,R1, R2)

```
Criminal(x1) <- American(x1), Weapon(y1), Sells(x1,y1,z1), Hostile(z1)
Sells(West,x2,Nono) <- Missile(x2), Owns(Nono,x2)
Weapon(x3) <- Missile(x3)
Hostile(x4, America) <- Enemy(x4, America)
Owns(Nono, M1)
Missile(M1)
American(West)
Enemy(Nono,America)
Query / To Prove:  Criminal(West)
```

In [10]:
from pyDatalog import pyDatalog
from pyDatalog.pyDatalog import assert_fact, retract_fact, ask, create_terms

pyDatalog.clear()
pyDatalog.create_terms('criminal, american, weapon, sells, hostile, owns, missile, enemy')
pyDatalog.create_terms('X, Y, Z')
pyDatalog.create_terms('nothing')

criminal(X) <= american(X) & weapon(Y) & sells(X,Y,Z) & hostile(Z, 'America') & nothing(X)
sells('West', X, 'Nono') <= missile(X) & owns('Nono', X)
weapon(X) <= missile(X)
hostile(X, 'America') <= enemy(X, 'America')
+owns('Nono', 'M1')
+missile('M1')
+american('West')
+enemy('Nono', 'America')
+nothing('foo')


In [11]:
criminal(X)

[]

In [ ]:
pyDatalog.create_terms('happy, healthy, wealthy, wise')
pyDatalog.create_terms('X')

happy(X) <= healthy(X), wealthy(X)
happy(X) <= wise(X)
+happy('Fred')
+healthy('Fred')
+wise('Fred')


In [ ]:
pyDatalog.ask('happy(X)').answers

### Sucky Policy

Agent takes care of macro actions and paths, and depleted battery
First try only one piece of gold

Best Move:
*  SUCK if at (r,c) and dirt at (r,c)
*  UNLOAD if at (1,1) and have gold
*  GOTO (1,1) if have gold
*  MINE if at (r,c) and gold at (r,c)
*  GOTO (r,c) gold at (r,c)



In [ ]:
from constants import ACTION_SUCK
create_terms('best_action', 'S', 'T')
best_action(ACTION_SUCK) <= location(S) and square_state((S, T)) and (T == DIRT)

In [ ]:
##  Build recon for a world
from constants import EAST

recon = {
    'height': 5,
    'width': 5,
    'position': (1,1),
    'heading': EAST,
    'gold': [(3,3)],
    'walls': [(0,0), (0,1), (0,2), (0,3), (0,4),
             (1,0), (1,4),
             (2,0), (2,4),
             (3,0), (3,4),
             (4,0), (4,1), (4,2), (4,3), (4,4),
             (2,2), (3,1)],
    'dirt': [(1,2)]
}

In [ ]:
from agents.agentworldmodel import AgentWorldModel
awm = AgentWorldModel()
awm.tell_recon(recon)

In [ ]:
awm.ask_free_squares()




World Model in Datalog
tell_percepts(dirt, bump) <= tell_percept_dirt(Dirt) and tell_percept_bump(Bump)
tell_recon()
tell_percepts()
tell_action()
ask_next_action()

predicates:
current_location(R,C)
square_state(R,C,state)
num_gold


In [ ]:
from pyDatalog import pyDatalog
from pyDatalog.pyDatalog import assert_fact, retract_fact, ask, create_terms, clear

In [ ]:
def replace(pred, newval):
    ans = pyDatalog.ask(f"{pred}(_)")
    if ans != None and len(ans.answers) > 0:
        oldval = ans.answers[0]
        retract_fact(pred, oldval)
    assert_fact(pred, newval)

def ask_pred(pred):
    return ask(f"{pred}(_)").answers[0][0]

def ask_square_state(sq):
    a = ask(f"square_state(({sq}, _))")
    if a == None:
        return None
    else:
        return a.answers[0][0]

In [ ]:
from constants import WALL, GOLD, DIRT, CLEAN
from constants import ACTION_FORWARD, ACTION_TURN_LEFT, ACTION_TURN_RIGHT, ACTION_MINE_GOLD, ACTION_UNLOAD_GOLD

create_terms('location, heading, square_state, num_gold, last_action')

def init_world_model():
    clear()
    assert_fact('num_gold', 0)
    assert_fact('last_action', None)
            
def tell_recon(recon):
    assert_fact('location', recon['position'])
    assert_fact('heading', recon['heading'])
    if 'walls' in recon:
        for square in recon['walls']:
            assert_fact('square_state', (square, "wall"))
    if 'gold' in recon:
        for square in recon['gold']:
            assert_fact('square_state', (square, "gold"))
    
def tell_last_action(action):
    assert_fact('last_action', action)
        
def tell_percepts(self, percepts):
    dirt = percepts.attributes['dirt']
    bump = percepts.attributes['bump']
    last_action = ask_pred('last_action')
    location = ask_pred('location')
    heading = ask_pred('heading')
    num_gold = ask_pred('num_gold')
    if last_action == ACTION_FORWARD and bump:
        assert_fact('square_state', (square_at_heading(location, heading), WALL))
    elif last_action == ACTION_FORWARD and not bump:
        newsquare = square_at_heading(location, heading)
        assert_fact('location', newsquare)
        assert_fact('square_state', (newsquare, DIRT if dirt else CLEAN))
    elif last_action == ACTION_TURN_LEFT:
        assert_fact('heading', heading_in_direction(heading, LEFT))
    elif last_action == ACTION_TURN_RIGHT:
        assert_fact('heading', heading_in_direction(heading, RIGHT))
    elif last_action == ACTION_MINE_GOLD and ask_square_state(location) == GOLD:
        assert_fact('square_state', (location, CLEAN))
        assert_fact('num_gold', num_gold+1)
    elif last_action == ACTION_UNLOAD_GOLD and location == (1,1):
        assert_fact('num_gold', num_gold-1)
    # For first step, last_action is None but want to set DIRT anyway
    if dirt:
        assert_fact('square_state', (location, DIRT))
    else:
        if self.ask_state_at(location) == UNKNOWN:
            assert_fact('square_state', (location, CLEAN))


In [ ]:
init_world_model()

In [ ]:
pyDatalog.ask('location(X)').answers

In [ ]:
ask_pred('heading')

In [ ]:
tell_recon(recon)

In [ ]:
ask(f"square_state((X, gold))").answers

In [ ]:
ask(f"square_state((X, wall))").answers

In [ ]:
tell_recon(recon)

In [ ]:
location(X)

In [ ]:
replace_location(2,3)

In [ ]:
pyDatalog.ask('location(X,Y)').answers

In [ ]:
tell_recon(recon)

In [ ]:
def dostuff():
    assert_fact("foo", 1)
    assert_fact("foo", 2)
    assert_fact("foo", 3)

In [ ]:
clear()
create_terms('foo', 'bar')
dostuff()

In [ ]:
foo(X)

In [ ]:
clear()
create_terms('square_state')
+square_state(((1,2), "foo"))


In [ ]:
ask_square_state((1,3))